In [74]:
import functools
from time import perf_counter
from kooplearn.datasets import LogisticMap
from kooplearn.models import KernelDMD
from kooplearn.data import traj_to_contexts
from kooplearn._src.utils import topk
from kooplearn._src.metrics import directed_hausdorff_distance
from sklearn.gaussian_process.kernels import RBF, Matern
from sklearn.model_selection import ParameterGrid
from scipy.spatial.distance import pdist

import numpy as np
import ml_confs
from sklearn.utils import resample

configs_dict = {
    'N': 20,
    'num_train': 20000, 
    'num_val': 1000,
    'num_test': 10000
}

configs = ml_confs.from_dict(configs_dict, register_jax_pytree=False)

# Adapted from https://realpython.com/python-timer/#creating-a-python-timer-decorator
def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = perf_counter()
        value = func(*args, **kwargs)
        toc = perf_counter()
        elapsed_time = toc - tic
        return value, elapsed_time

    return wrapper_timer

In [45]:
logistic = LogisticMap(N=configs.N, rng_seed=0)
# Data pipeline
sample_traj = logistic.sample(
    0.5, configs.num_train + configs.num_val + configs.num_test
)

dataset = {
    "train": sample_traj[: configs.num_train],
    "validation": sample_traj[configs.num_train : configs.num_train + configs.num_val],
    "test": sample_traj[configs.num_train + configs.num_val :],
}

ref_eigs = logistic.eig()
top_eigs = topk(np.abs(ref_eigs), 3)
ref_eigs = ref_eigs[top_eigs.indices]

In [61]:
train_traj = resample(dataset['train'], replace=False, n_samples=1000, random_state=0)
train_ctx = traj_to_contexts(train_traj)
test_ctx = traj_to_contexts(dataset['test'])
val_ctx = traj_to_contexts(dataset['validation'])

In [66]:
num_ls = 11
ls_quantiles = np.linspace(0.05, 0.95, num_ls)
ls_grid = np.quantile(pdist(train_traj), ls_quantiles)
reg_grid = np.logspace(-5, 0, 11)
hp_grid = ParameterGrid({'ls': ls_grid, 'reg': reg_grid})

In [69]:
def train_and_eval(ls, reg, kernel, svd_solver, n_oversamples = 20, rank = 8, rng_seed = 0):
    kernel = RBF(length_scale=ls)
    model = KernelDMD(kernel = kernel, rank = rank, tikhonov_reg = reg, svd_solver=svd_solver, n_oversamples=n_oversamples, rng_seed=rng_seed)
    timed_fit = timer(model.fit)
    model, fit_time = timed_fit(train_ctx)
    val_risk = model.risk(val_ctx)
    return model, val_risk, fit_time

In [83]:
best_risk = np.inf
best_vals = None
best_model = None

timings = []
risks = []
hp_params = list(hp_grid)

for hp in hp_grid:
    model, val_risk, fit_time = train_and_eval(hp['ls'], hp['reg'], RBF, 'randomized')
    timings.append(fit_time)
    risks.append(val_risk)
    if val_risk < best_risk:
        best_risk = val_risk
        best_vals = hp
        best_model = model

Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD mod

In [82]:
best_risk

0.1270142643714789

In [19]:
ls = 1.0 # To be tuned
reg = 1e-3 # To be tuned

R3 = KernelDMD(kernel = kernel, rank = 8, tikhonov_reg = reg, svd_solver='arnoldi')
R4 = KernelDMD(kernel = kernel, rank = 8, tikhonov_reg = reg, svd_solver='randomized', n_oversamples=20, rng_seed=0)

In [25]:
R3 = R3.fit(train_ctx)
R4 = R4.fit(train_ctx)

The computed projector is not real. The Kernel matrix is severely ill-conditioned.
The numerical rank of the projector is smaller than the selected rank (8). 2 degrees of freedom will be ignored.


Fitted KernelDMD model. Lookback length set to 1
Fitted KernelDMD model. Lookback length set to 1


In [28]:
R3.risk()

0.07091020631203315

In [29]:
R4.risk()

0.07149243053042675

In [39]:
R3_eigs = R3.eig()
top_eigs = topk(np.abs(R3_eigs), 3)
R3_eigs = R3_eigs[top_eigs.indices]

R4_eigs = R4.eig()
top_eigs = topk(np.abs(R4_eigs), 3)
R4_eigs = R4_eigs[top_eigs.indices]

In [41]:
R3_hausdorff = directed_hausdorff_distance(R3_eigs, ref_eigs)
R4_hausdorff = directed_hausdorff_distance(R4_eigs, ref_eigs)

In [42]:
R3_hausdorff

0.33647241287080293

In [43]:
R4_hausdorff

0.44529769200695607